In [2]:
DATA_IN = "../data/raw/rees46/"
DATA_OUT = "../data/intermediate/rees46/"

import findspark
findspark.init()

In [3]:
#import pyspark 
from pyspark.sql import SparkSession 

# later on - probably add dtypes in load phase
from pyspark.sql.types import StringType, IntegerType,\
    DoubleType, StructType, StructField, DateType

In [4]:
#sc = pyspark.SparkContext()
ss = SparkSession.builder.getOrCreate()
# terminate # set parames # get or create
ss.sparkContext.setLogLevel("ERROR")
ss.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
#ss.sparkContext.getConf().getAll()

22/03/15 17:25:05 WARN Utils: Your hostname, mf-H110M-S2H resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface enx00d2b176ce73)
22/03/15 17:25:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/15 17:25:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def get_sdf_info(sdf, name=""):
    rc = sdf.count(); cc = len(sdf.columns) # note - wrap this into a fuc
    print("The {} dataset has shape of ({},{}) and following dtypes\n{}".\
        format(name, rc, cc, sdf.dtypes))

In [6]:
import glob
for f in glob.glob(DATA_IN+"*.gz"):
    if "events" not in locals():
        events = ss.read.csv(f,header=True, inferSchema=True, nanValue="null")
    else:
        events.union(ss.read.csv(f,header=True, inferSchema=True, nanValue="null"))
#events = ss.read.csv(glob.glob(DATA_DIR+"*.gz")[2],header=True, inferSchema=True)
get_sdf_info(events)

The  dataset has shape of (67542878,9) and following dtypes
[('event_time', 'string'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_id', 'int'), ('user_session', 'string')]


In [7]:
# keep interactions for users with >=10 trans
from pyspark.sql.functions import col, countDistinct
users_to_keep = events.filter(col("event_type")=="purchase").\
    groupBy("user_id").agg(countDistinct("user_session").\
        alias("n_transactions")).filter(col("n_transactions")>=0).select(col("user_id"))
events = events.join(users_to_keep, ["user_id"], "inner")
get_sdf_info(events)

The  dataset has shape of (25219436,9) and following dtypes
[('user_id', 'int'), ('event_time', 'string'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_session', 'string')]


In [8]:
# convert dates
from pyspark.sql.functions import to_timestamp
events = events.withColumn("event_time", to_timestamp(col("event_time")))
get_sdf_info(events)

The  dataset has shape of (25219436,9) and following dtypes
[('user_id', 'int'), ('event_time', 'timestamp'), ('event_type', 'string'), ('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string'), ('price', 'double'), ('user_session', 'string')]


In [9]:
# carve-out product-cat info as it is duplicated
products = events.select(["product_id", "category_id", "category_code", "brand"]).dropDuplicates()
events = events.select(["event_time", "event_type", "user_id", "product_id", "price","user_session"])
get_sdf_info(products, "products")
get_sdf_info(events, "events")

The products dataset has shape of (180907,4) and following dtypes
[('product_id', 'int'), ('category_id', 'bigint'), ('category_code', 'string'), ('brand', 'string')]


The events dataset has shape of (25219436,6) and following dtypes
[('event_time', 'timestamp'), ('event_type', 'string'), ('user_id', 'int'), ('product_id', 'int'), ('price', 'double'), ('user_session', 'string')]


In [10]:
# try to save both of the datasets to gz
import os
if not os.path.exists(DATA_OUT):
    os.makedirs(DATA_OUT)
events.write.csv(DATA_OUT+"events", compression="gzip")
products.write.csv(DATA_OUT+"products", compression="gzip")